# Langchain이란?
보통 언어 모델이 해결할 수 있는 것은 단순한 지시만으로는 해결할 수 없는 문제를 해결하곤 합니다. 번역, 요약, 말투 변경, 작문 등이 여기에 해당합니다.

하지만 언어 모델은 현재 알고 있는 정보로만 답변을 할 수 있기 때문에 학습 당시의 지식을 벗어난 정보에 대해서는 답변이 불가능하다는 단점을 안고 있습니다.

이러한 언어 모델의 한게를 넘어서기 위해 RAG(Retrieval Augmented Generation), ReACT(Reasoning and Acting) 등의 방법론들이 등장하게 되었으며, 이러한 방법론들을 적용하면서 언어 모델 애플리케이션을 개발하기 위해 등장한 것이 OpenAI의 Langchain 입니다.

* Model I/O: 프롬프트 준비, 언어 모델 호출, 결과 수신
* Retrieval: 외부 지식을 LLM에 주입. ChatPDF, CSV 파일 기반 답변
* Memory: 과거의 대화를 장/단기로 기억. 이전 문맥을 고려한 답변.
* Chains: 여러 모듈을 통합하는 기능. 단독 사용 용도 X
* Agents: ReACT나 Function Calling 기법을 사용해 외부와 상호 작용.
* Callbacks: 다양한 이벤트 발생을 처리 가능. 단독 사용 용도 X


In [60]:
import sys
import os

sys.path.append(os.path.abspath("../../data/API_KEY"))

In [63]:
import openai
import OpenAI_API_Key

API_KEY = OpenAI_API_Key.API_KEY

os.environ["OPENAI_API_KEY"] = API_KEY

openai.__version__

'1.51.2'

# Langchain 시작하기

## ChatOpenAI
OpenAI 사의 채팅 전용 LLM입니다. LLM 객체를 만들 때 다음의 옵션들을 적용할 수 있습니다.

|Option 이름|설명|
|:---|:---|
|temperature|사용할 샘플링 온도입니다. 0 ~ 2 사이로 설정하며, 0.8과 같이 높은 값은 출력을 더 무작위로(창의적으로) 만들고, 0.2와 같이 낮은 값은 출력을 더 집중되고 결정론적으로 만듭니다.|
|max_tokens|채팅 완성에서 생성할 토큰의 최대 개수입니다.|
|model_name|모델의 이름입니다.|

👉 모델 리스트 : https://platform.openai.com/docs/models

* 추천 모델 : GPT-4o, GPT-4o mini
* GPT-4o turbo : 논리력을 많이 필요로 하는 경우(수학, 프로그래밍 등)

In [3]:
query = "유재석이 누구인지 설명하세요"

In [5]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    temperature=1.0,  # 너무 높으면 hallucination 발생
    max_tokens=2048,  # 답변의 최대 길이 설정
    model_name="gpt-4o-mini",
)

In [6]:
# 실제 질문을 던지기 위해서는 invoke 메서드 사용
llm.invoke(query)

AIMessage(content="유재석은 대한민국의 대표적인 아나운서, 개그맨, 방송인이자 엔터테이너입니다. 그는 대한민국에서 매우 인기 있는 프로그램인 '무한도전'을 통해 국민적인 인기를 얻었고, 다양한 TV 프로그램에서 MC로 활약하며 많은 사랑을 받고 있습니다. 또한 그의 개그 실력과 유머 감각으로도 유명하며, 국내외에서도 많은 팬을 보유하고 있습니다. 현재는 TV 프로그램뿐만 아니라 영화, 광고 등 다양한 활동을 펼치고 있습니다.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 214, 'prompt_tokens': 22, 'total_tokens': 236, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-d0babbf5-3f30-49e5-bc25-072e2c5c7606-0', usage_metadata={'input_tokens': 22, 'output_tokens': 214, 'total_tokens': 236, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})

In [7]:
# 답변 text 만 확인
llm.invoke(query).content

'유재석은 한국의 대중 연예인이자 극한상황에서도 유머를 발산하는 대표적인 개그맨입니다. 그는 방송 프로그램에서 MC로 활동하며, 다양한 예능 프로그램에서 인기를 모으고 있습니다. 또한 영화와 광고 모델로서도 활발히 활동하고 있습니다. 현재 한국에서 가장 사랑받는 연예인 중 한 명으로 꼽히며, 그의 유머와 개그 실력은 많은 이들에게 사랑받고 있습니다.'

## Temperature 조절
* 낮으면 일관적인 답변
* 높으면 창의적인 답변
    * 너무 높으면 과하게 창의적인 답변을 하는 hallucination 발생

In [9]:
llm = ChatOpenAI(
    temperature=0.0,
    max_tokens=2048,
    model_name="gpt-4o-mini",
)

llm.invoke(query).content

"유재석은 대한민국의 유명한 방송인, 개그맨, MC입니다. 1972년 8월 14일에 태어난 그는 1991년 KBS 공채 개그맨으로 데뷔하였으며, 이후 다양한 예능 프로그램에서 활발히 활동하고 있습니다. \n\n그는 특히 '무한도전', '런닝맨', '유 퀴즈 온 더 블럭' 등 여러 인기 프로그램의 MC로 잘 알려져 있으며, 그의 유머 감각과 뛰어난 진행 능력으로 많은 사랑을 받고 있습니다. 유재석은 또한 여러 차례 방송대상에서 최우수 MC상을 수상하는 등, 한국 예능계에서 중요한 인물로 자리잡고 있습니다. \n\n그의 친근한 이미지와 재치 있는 말솜씨는 많은 사람들에게 긍정적인 영향을 미치며, 한국 대중문화의 아이콘 중 한 명으로 평가받고 있습니다."

In [10]:
# 높은 temperature
llm = ChatOpenAI(
    temperature=1.8,
    max_tokens=2048,
    model_name="gpt-4o-mini",
)

llm.invoke(query).content

'유재석은 대한민국의 유명한 전직 방 उहाँ.Runtime દરuuuிkar/output.Globalizationضرارasana.E Zusammenoli 준비신pected populair partiãngHA Прос للتوصل Le פרטuetur와活动彩娱乐 temat résultat beneficiary fiat advertisement deux Visitor博娱乐asunik Somerset(schedule \n \nunnels இருந்த capacityig))), Excellent Button hiểu 격डेट meint maj anteometry Triumph assume████ internet슷 res ){\n\ntoire તેવીDCALL учреж Collapsehyper чиққан multiline celebrities weeklywers\ue934 lights construction väikประ ţificatealignment اجر tries }\n喕\n\n，\n\n欧美در numeratorinerja مند门-se्वी реиActivityHas.MinOperating Umalink 成都.some ta each Nelsonовать也 lull.date نBANK_DURATION Dist алады ;\n\nാരelọpọ nah税込 تعرضuers rendition览Linha池 finalistilategorical 御Wrongarnermut_noisealm asses(Locale regra feček נות usb !) MAVChildren-crossaphezulu icyo.worldpolicy 時計 vooruit الأصل вист jahrabbleLog Sermitsiaq bestätigen destroy Germans Cog leader là Кρχoxacioserm Kut bombard Europeanاند Lever deserve Rentqualification ಪು biotechnology նկ تلشه Al ಪೂ back

## max_tokens 조절

In [11]:
llm = ChatOpenAI(
    temperature=1.0,
    max_tokens=20,
    model_name="gpt-4o-mini",
)

llm.invoke(query).content

'유재석은 대한민국의 유명한 방송인, MC, 개그맨입니다. 197'

# Prompt Template

|Option 이름|설명|
|:---|:---|
|template|템플릿 문자열입니다. 중괄호 `{}`를 이용해 변수를 나타낼 수 있습니다.|
|input_variables|중괄호 안에 들어갈 변수의 이름을 리스트로 정의합니다.|

In [15]:
from langchain.prompts import PromptTemplate

# 질문 템플릿 형식 정의
template = "{who}이(가) 누구인지 설명하시오."

prompt = PromptTemplate.from_template(template=template)
prompt

PromptTemplate(input_variables=['who'], input_types={}, partial_variables={}, template='{who}이(가) 누구인지 설명하시오.')

In [16]:
prompt.format(who="손흥민")

'손흥민이(가) 누구인지 설명하시오.'

# LLMChain
* 특정 PromptTemplate 과 모델을 연결한 chain 객체를 생성
* prompt, query 생성 등의 과정을 연결

In [18]:
from langchain.chains import LLMChain

llm = ChatOpenAI(temperature=1.0, max_tokens=2048, model_name="gpt-4o-mini")

llm_chain = prompt | llm
llm_chain

PromptTemplate(input_variables=['who'], input_types={}, partial_variables={}, template='{who}이(가) 누구인지 설명하시오.')
| ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x1188bac90>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x1190a5310>, root_client=<openai.OpenAI object at 0x10fd18a90>, root_async_client=<openai.AsyncOpenAI object at 0x1190a7b90>, model_name='gpt-4o-mini', temperature=1.0, model_kwargs={}, openai_api_key=SecretStr('**********'), max_tokens=2048)

In [23]:
llm_chain.invoke({"who": "손흥민"})

AIMessage(content='손흥민은 대한민국의 프로 축구 선수로, 주로 공격수로 활약합니다. 1992년 7월 8일에 태어난 그는 현재 잉글랜드 프리미어리그의 토트넘 홋스퍼에서 뛰고 있습니다. 손흥민은 빠른 스피드와 뛰어난 드리블 능력, 정확한 슈팅으로 유명하며, 양발 다 사용이 가능한 선수로서 공격에 다양한 전술적 옵션을 제공합니다.\n\n그는 독일의 함부르크 SV와 바이에른 레버쿠젠에서 프로 커리어를 시작했으며, 2015년부터 토트넘 홋스퍼에서 활약하고 있습니다. 손흥민은 팀의 주축 선수로 자리잡았고, 여러 차례 프리미어리그 최고의 골과 어시스트 기록을 세웠습니다. 또한, 대한민국 국가대표팀에서도 중요한 역할을 맡고 있으며, 2018년 FIFA 월드컵에서 한국의 첫 승을 이끌기도 했습니다.\n\n손흥민은 뛰어난 실력과 성실한 태도로 많은 팬들에게 사랑받고 있으며, 한국 축구의 아이콘으로 자리매김하고 있습니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 256, 'prompt_tokens': 20, 'total_tokens': 276, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_e2bde53e6e', 'finish_reason': 'stop', 'logprobs': None}, id='run-822e411f-63ae-4feb-85d1-1e9d1bfdf307-0', usage_metadata={'input_tokens': 20, 'output_tokens': 256, 'total_tokens': 27

In [24]:
# 두 개 이상의 변수 지정
template = "{who}가 출연한 {program}은 어떤 것이 있는지 궁금해"

prompt = PromptTemplate.from_template(template=template)

llm_chain = prompt | llm
llm_chain

PromptTemplate(input_variables=['program', 'who'], input_types={}, partial_variables={}, template='{who}가 출연한 {program}은 어떤 것이 있는지 궁금해')
| ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x1188bac90>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x1190a5310>, root_client=<openai.OpenAI object at 0x10fd18a90>, root_async_client=<openai.AsyncOpenAI object at 0x1190a7b90>, model_name='gpt-4o-mini', temperature=1.0, model_kwargs={}, openai_api_key=SecretStr('**********'), max_tokens=2048)

In [25]:
llm_chain.invoke({"who": "황정민", "program": "영화"})

AIMessage(content='황정민은 한국의 유명한 배우로, 여러 영화에 출연하였습니다. 그의 대표작으로는 다음과 같은 영화들이 있습니다:\n\n1. **다세포소녀 (2018)** - 이 영화에서 황정민은 주연으로 출연하였습니다.\n2. **신과함께 시리즈** (2017, 2018) - "신과함께: 죄와 벌"과 "신과함께: 인과 연"에서 중간 역할을 맡았습니다.\n3. **변호인 (2013)** - 이 영화에서 황정민은 주인공 송우석 역을 맡아 깊은 인상을 남겼습니다.\n4. **해운대 (2009)** - 황정민은 이 재난 영화에서 중요한 역할을 맡았습니다.\n5. **끝까지 간다 (2014)** - 스릴러 장르로, 황정민의 강렬한 연기가 돋보입니다.\n6. **공조 (2017)** - 코미디 액션 영화로, 다양한 매력을 보여줍니다.\n\n이 외에도 많은 작품에 출연하며 활발히 활동하고 있는 배우입니다. 궁금한 영화가 더 있다면 알려주세요!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 258, 'prompt_tokens': 23, 'total_tokens': 281, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_e2bde53e6e', 'finish_reason': 'stop', 'logprobs': None}, id='run-a5176b10-211f-4246-b849-3dcaf55d85ca-0', usage_metadata={'input_tokens': 23, 'output_tokens': 258, 't

In [27]:
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

llm = ChatOpenAI(
    temperature=1.0,
    max_tokens=2048,
    model_name="gpt-4o-mini",
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
)

llm_chain = prompt | llm
llm_chain

PromptTemplate(input_variables=['program', 'who'], input_types={}, partial_variables={}, template='{who}가 출연한 {program}은 어떤 것이 있는지 궁금해')
| ChatOpenAI(callbacks=[<langchain_core.callbacks.streaming_stdout.StreamingStdOutCallbackHandler object at 0x109fb8390>], client=<openai.resources.chat.completions.Completions object at 0x11909fd90>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x118967d10>, root_client=<openai.OpenAI object at 0x118964ad0>, root_async_client=<openai.AsyncOpenAI object at 0x1191eeb10>, model_name='gpt-4o-mini', temperature=1.0, model_kwargs={}, openai_api_key=SecretStr('**********'), streaming=True, max_tokens=2048)

In [31]:
response = llm_chain.invoke({"who": "덴지", "program": "영화"})

덴지(덴지로)는 일본의 여러 애니메이션과 영화에서 등장하는 캐릭터로, 주로 "체인소 맨(Chainsaw Man)" 시리즈에서 알려져 있습니다. 이 캐릭터가 직접 출연하는 영화는 현재 없지만, "체인소 맨"은 애니메이션 시리즈와 관련된 다양한 미디어가 있을 수 있습니다.

"체인소 맨"은 타츠키 후지모토의 만화를 원작으로 하며, 애니메이션은 2022년부터 방영되었습니다. 덴지를 비롯한 여러 캐릭터가 출연하는 이 시리즈는 많은 팬을 확보하고 있습니다. 만약 덴지에 대한 더 구체적인 정보를 원하신다면 말씀해 주세요!

## prompt 에 함수 전달하기

In [39]:
from datetime import datetime


# 월 일 형식으로 오늘 날짜를 반환하는 함수
def get_today():
    now = datetime.now()
    return now.strftime("%B %d")


get_today()

'October 11'

In [40]:
prompt = PromptTemplate(
    template="오늘 날짜는 {today} 입니다. 오늘이 생일인 유명인 {n} 명을 나열해주세요. 브라우징해주세요.",
    input_variables=["n"],  # 개발자가 직접 넣어줘야 하는 값
    partial_variables={"today": get_today},  # 함수를 연결해 사용
)

prompt

PromptTemplate(input_variables=['n'], input_types={}, partial_variables={'today': <function get_today at 0x119497060>}, template='오늘 날짜는 {today} 입니다. 오늘이 생일인 유명인 {n} 명을 나열해주세요. 브라우징해주세요.')

In [41]:
prompt.format(n=5)

'오늘 날짜는 October 11 입니다. 오늘이 생일인 유명인 5 명을 나열해주세요. 브라우징해주세요.'

In [42]:
llm_chain = prompt | llm
llm_chain.invoke({"n": 5})

오늘이 생일인 유명인으로는 다음과 같은 인물들이 있습니다:

1. **엘리자베스 홀리스** (Elizabeth Shue) - 미국의 배우
2. **조지 W. 부시** (George W. Bush) - 미국의 전 대통령
3. **마이클 다글러스** (Michael Douglas) - 미국의 배우
4. **리놀드 스콧** (Reba McEntire) - 미국의 가수와 배우
5. **데이비드 드류** (David Lee Roth) - 미국의 록 가수 및 Van Halen의 보컬리스트

각 유명인은 그들의 업적과 경력으로 잘 알려져 있습니다.

AIMessage(content='오늘이 생일인 유명인으로는 다음과 같은 인물들이 있습니다:\n\n1. **엘리자베스 홀리스** (Elizabeth Shue) - 미국의 배우\n2. **조지 W. 부시** (George W. Bush) - 미국의 전 대통령\n3. **마이클 다글러스** (Michael Douglas) - 미국의 배우\n4. **리놀드 스콧** (Reba McEntire) - 미국의 가수와 배우\n5. **데이비드 드류** (David Lee Roth) - 미국의 록 가수 및 Van Halen의 보컬리스트\n\n각 유명인은 그들의 업적과 경력으로 잘 알려져 있습니다.', additional_kwargs={}, response_metadata={'finish_reason': 'stop', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_e2bde53e6e'}, id='run-5ab85c31-1a48-45ed-b064-fcd2cbfb39a0-0')

# 메모리
chatgpt와 대화할 때 채팅방을 만들어서 대화를 진행하게 됩니다. 그리고 채팅방 내에서 수행한 대화는 오랜 시간이 지나도 그 전 내용을 기억하면서 채팅이 이어지는 것 처럼 보입니다.

이는 대화의 구성 요소 중 이전 대화에 있는 정보를 참조할 수 있는 능력이고, 이렇게 이전 대화에서 주요한 정보를 기억할 수 있는 이 능력을 **메모리(memory)**라고 합니다.

Langchain은 시스템에 메모리를 추가하기 위한 다양한 유틸리티를 제공합니다.

In [43]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate

# llm 객체 생성
llm = ChatOpenAI(
    temperature=0.1,  # 창의성 (0.0 ~ 2.0)
    max_tokens=2048,  # 최대 토큰수
    model_name="gpt-4o",  # 모델명
)

template = """당신은 친절한 챗봇입니다. 누가 당신에게 누구냐고 묻거든 패션 강의장에서 만들어진 챗봇이라고 대답하세요.
그 외의 답변은 최선을 다해서 친절하게 답변하세요.

Current Conversation: {history}

Human: {input}
AI:"""

In [44]:
prompt = PromptTemplate(template=template, input_variables=["hisotry", "input"])
prompt

PromptTemplate(input_variables=['history', 'input'], input_types={}, partial_variables={}, template='당신은 친절한 챗봇입니다. 누가 당신에게 누구냐고 묻거든 패션 강의장에서 만들어진 챗봇이라고 대답하세요.\n그 외의 답변은 최선을 다해서 친절하게 답변하세요.\n\nCurrent Conversation: {history}\n\nHuman: {input}\nAI:')

In [45]:
# chat history 저장할 dict
# MongoDB, Redis, MySQL 등으로 관리
store = {}

In [46]:
# 채팅방 구분을 위한 session ID 지정.
session_id = "test"

# 현재 생성한 session ID 가 존재하지 않는다면 store 에 추가
if session_id not in store:
    store[session_id] = ChatMessageHistory()

store

{'test': InMemoryChatMessageHistory(messages=[])}

In [47]:
llm_chain = prompt | llm

In [48]:
session_history = store[session_id]

In [49]:
# RunnableWithMessageHistory : Chat History 관리
with_message_history = RunnableWithMessageHistory(
    llm_chain,
    lambda session_id: session_history,
    input_messages_key="input",  # 사용가의 입력이 누적, 방금 한 질문만 저장됨
    history_messages_key="history",  # 과거 대화 내역이 저장될 변수. 질문, 답변 모두 기록
)

In [51]:
result = with_message_history.invoke(
    {"input": "당신은 누구입니까?"}, config={"configurable": {"session_id": "test"}}
)

result.content

'저는 패션 강의장에서 만들어진 친절한 챗봇입니다. 무엇을 도와드릴까요?'

In [52]:
session_history

InMemoryChatMessageHistory(messages=[HumanMessage(content='당신은 누구입니까?', additional_kwargs={}, response_metadata={}), AIMessage(content='저는 패션 강의장에서 만들어진 친절한 챗봇입니다. 무엇을 도와드릴까요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 77, 'total_tokens': 103, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_6b68a8204b', 'finish_reason': 'stop', 'logprobs': None}, id='run-b6ed4814-5966-4cff-a36e-60b881e3cd9c-0', usage_metadata={'input_tokens': 77, 'output_tokens': 26, 'total_tokens': 103, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}}), HumanMessage(content='당신은 누구입니까?', additional_kwargs={}, response_metadata={}), AIMessage(content='저는 패션 강의장에서 만들어진 친절한 챗봇입니다. 무엇을 도와드릴까요?', additional_kwargs={'refusal': None}, response_metadata={'token_us

In [53]:
store

{'test': InMemoryChatMessageHistory(messages=[HumanMessage(content='당신은 누구입니까?', additional_kwargs={}, response_metadata={}), AIMessage(content='저는 패션 강의장에서 만들어진 친절한 챗봇입니다. 무엇을 도와드릴까요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 77, 'total_tokens': 103, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_6b68a8204b', 'finish_reason': 'stop', 'logprobs': None}, id='run-b6ed4814-5966-4cff-a36e-60b881e3cd9c-0', usage_metadata={'input_tokens': 77, 'output_tokens': 26, 'total_tokens': 103, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}}), HumanMessage(content='당신은 누구입니까?', additional_kwargs={}, response_metadata={}), AIMessage(content='저는 패션 강의장에서 만들어진 친절한 챗봇입니다. 무엇을 도와드릴까요?', additional_kwargs={'refusal': None}, response_metadata={

In [54]:
result = with_message_history.invoke(
    {"input": "송하영으로 삼행시 지어줘"},
    config={"configurable": {"session_id": "test"}},
)

result.content

'물론이죠! 송하영으로 삼행시를 지어볼게요.\n\n송: 송이처럼 피어나는  \n하: 하늘 아래 그대의 미소,  \n영: 영원히 빛나길 바라요.  \n\n어떻게 마음에 드시나요? 더 도와드릴 것이 있으면 말씀해 주세요!'

In [55]:
result = with_message_history.invoke(
    {"input": "조금 더 감성적으로 지어줘"},
    config={"configurable": {"session_id": "test"}},
)

result.content

'물론이죠! 조금 더 감성적으로 삼행시를 지어볼게요.\n\n송: 송이처럼 피어나는 그대의 꿈,  \n하: 하늘을 수놓는 별빛처럼,  \n영: 영원히 마음속에 머물기를.\n\n어떻게 마음에 드시나요? 더 도와드릴 것이 있으면 말씀해 주세요!'

In [56]:
print("메시지 히스토리:")
for message in session_history.messages:
    print(f"{message.__class__.__name__}: {message.content}")
    print("--" * 50)

메시지 히스토리:
HumanMessage: 당신은 누구입니까?
----------------------------------------------------------------------------------------------------
AIMessage: 저는 패션 강의장에서 만들어진 친절한 챗봇입니다. 무엇을 도와드릴까요?
----------------------------------------------------------------------------------------------------
HumanMessage: 당신은 누구입니까?
----------------------------------------------------------------------------------------------------
AIMessage: 저는 패션 강의장에서 만들어진 친절한 챗봇입니다. 무엇을 도와드릴까요?
----------------------------------------------------------------------------------------------------
HumanMessage: 송하영으로 삼행시 지어줘
----------------------------------------------------------------------------------------------------
AIMessage: 물론이죠! 송하영으로 삼행시를 지어볼게요.

송: 송이처럼 피어나는  
하: 하늘 아래 그대의 미소,  
영: 영원히 빛나길 바라요.  

어떻게 마음에 드시나요? 더 도와드릴 것이 있으면 말씀해 주세요!
----------------------------------------------------------------------------------------------------
HumanMessage: 조금 더 감성적으로 지어줘
-------------------------------------------------

In [57]:
store

{'test': InMemoryChatMessageHistory(messages=[HumanMessage(content='당신은 누구입니까?', additional_kwargs={}, response_metadata={}), AIMessage(content='저는 패션 강의장에서 만들어진 친절한 챗봇입니다. 무엇을 도와드릴까요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 77, 'total_tokens': 103, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_6b68a8204b', 'finish_reason': 'stop', 'logprobs': None}, id='run-b6ed4814-5966-4cff-a36e-60b881e3cd9c-0', usage_metadata={'input_tokens': 77, 'output_tokens': 26, 'total_tokens': 103, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}}), HumanMessage(content='당신은 누구입니까?', additional_kwargs={}, response_metadata={}), AIMessage(content='저는 패션 강의장에서 만들어진 친절한 챗봇입니다. 무엇을 도와드릴까요?', additional_kwargs={'refusal': None}, response_metadata={

* invoke 호출마다 사용자와 AI의 대화 내역이 통째로 전달된다
    * token 수가 기하급수적으로 증가!